In [176]:
# This notebook augments a given dataset.

In [1]:
import config
import importlib
importlib.reload(config)

<module 'config' from '/home/msc_student/vox2vox/config.py'>

In [178]:
# Installs

#!pip install elasticdeform
#!pip install tensorflow_addons
#!pip install nibabel
#!pip install matplotlib
#!pip install sklearn

In [179]:
# Imports

# System imports
import os
import glob
import concurrent.futures

# Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import apply_affine_transform
from tensorflow.keras.utils import to_categorical

# Numerical calculations
import numpy as np
from scipy.ndimage.interpolation import affine_transform
import elasticdeform as ed

# Own imports
import scan_loader
from data_generator import DataGenerator

<module 'scan_loader' from '/home/msc_student/vox2vox/scan_loader.py'>

In [180]:
# GPU setup

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = conf.gpu

allow_multi_gpu = True
tf_version = 2

if tf_version == 2 and allow_multi_gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print("Num GPUs Available: ", len(gpus))
    if gpus:
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)

Num GPUs Available:  1


In [181]:
# Finds paths to all MRT-scans in the dataset folder.

t1_list    = sorted(glob.glob(conf.dataset_mask_train + '*t1.nii.gz'))
t2_list    = sorted(glob.glob(conf.dataset_mask_train + '*t2.nii.gz'))
t1ce_list  = sorted(glob.glob(conf.dataset_mask_train + '*t1ce.nii.gz'))
flair_list = sorted(glob.glob(conf.dataset_mask_train + '*flair.nii.gz'))
seg_list   = sorted(glob.glob(conf.dataset_mask_train + '*seg.nii.gz'))

data = []
for i in range(len(t1_list)):
    data.append((t1_list[i], t2_list[i], t1ce_list[i], flair_list[i], seg_list[i]))

In [182]:
# Defines data augmentation functions

# Flips a MRT-image on a random plane
def flip3D(X, y):
    choice = np.random.randint(3)
    print(f"Flip: {choice}")
    
    if choice == 0: # flip on x
        X_flip, y_flip = X[::-1, :, :, :], y[::-1, :, :]
    if choice == 1: # flip on y
        X_flip, y_flip = X[:, ::-1, :, :], y[:, ::-1, :]
    if choice == 2: # flip on z
        X_flip, y_flip = X[:, :, ::-1, :], y[:, :, ::-1]
        
    return X_flip, y_flip

# Rotates a MRT-image randomly by up to 31°
def rotation3D(X, y):
    # TODO: Doesn't this only turn into positive direction? 
    # Wouldn't np.random.randint(-31,31,size=3) make more sense?
    alpha, beta, gamma = np.random.randint(0, 31, size=3)/180*np.pi
    print(f"Rotation: {alpha}, {beta}, {gamma}")
    
    # Calcualtes rotation matrices
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(alpha), -np.sin(alpha)],
                   [0, np.sin(alpha), np.cos(alpha)]])
    
    Ry = np.array([[np.cos(beta), 0, np.sin(beta)],
                   [0, 1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    
    Rz = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                   [np.sin(gamma), np.cos(gamma), 0],
                   [0, 0, 1]])
    
    # Combines rotation matrices into one rotation matrix
    # TODO: Take center of image into account.
    R = np.dot(np.dot(Rx, Ry), Rz)
    #t = np.array([[120,120,77]]).T
    #Rt = np.hstack((R,t))
    #Rt = np.linalg.inv(Rt)
    
    # Performs rotation
    X_rot = np.empty_like(X)
    backgrounds = X.min(axis=(0,1,2))
    
    for channel in range(X.shape[-1]):
        X_rot[:,:,:,channel] = affine_transform(X[:,:,:,channel], R, order=0, cval=backgrounds[channel])
        
    y_rot = affine_transform(y, R, order=0, mode="nearest")
    
    return X_rot, y_rot

### NEW AUGMENTATION TECHNIQUES
def brightness(X, y):
    """
    Changing the brighness of a image using power-law gamma transformation.
    Gain and gamma are chosen randomly for each image channel.
    
    Gain chosen between [0.9 - 1.1]
    Gamma chosen between [0.9 - 1.1]
    
    new_im = gain * im^gamma
    """
    print("Brightness")
    
    factor = 0.2
    
    X_new = np.zeros(X.shape)
    for c in range(X.shape[-1]):
        im = X[:,:,:,c]        
        gain, gamma = (1-factor) + np.random.random_sample(2,) * factor * 2.0
        im_new = np.sign(im)*gain*(np.abs(im)**gamma)
        X_new[:,:,:,c] = im_new 
    
    return X_new, y

def elastic(X, y):
    """
    Elastic deformation on a image and its target
    """  
    
    factor = np.random.uniform(0.0, 4.0)
    print(f"Elastic: {factor}")
  
    # Randomly transforms the image elastically.
    # Parts of the image that would be empty are set to +1000 so that they can be filled with the appropriate background later.
    [Xel, yel] = ed.deform_random_grid([X, y], sigma=factor, axis=[(0, 1, 2), (0, 1, 2)], order=[0, 0], cval=1000)
    
    # Empty parts of the segmentation contain no tumor.
    yel[yel == 1000] = 0
    
    for channel in range(Xel.shape[-1]):
        
        Xchannel = Xel[:,:,:,channel]
        
        # Sets all empty areas of the channel to the background intensity (minimum).
        Xchannel[Xchannel == 1000] = Xchannel.min()
        
        Xel[:,:,:,channel] = Xchannel
    
    return Xel, yel

def noise(X, y):
    """
    Adds random noise to the image.
    The noise has 1% magnitude of the average intensity of the scan.
    """
    
    brain = X[X > X.min()]
    noise_intensity = brain.mean() * 0.05
    noise_intensity *= np.random.random()
    
    print(f"Noise: {noise_intensity}")
    
    noise = (np.random.random(X.shape)-0.5) * 2.0 * noise_intensity
    
    Xnoise = X + noise
    return Xnoise, y

def contrast(X, y):
    """
    Changes the contrast of the image. Based on tf.image.adjust_contrast(X, contrast_factor)
    See: https://www.tensorflow.org/api_docs/python/tf/image/adjust_contrast
    Formula: contrasted_img = (img - mean) * contrast_factor + mean
    """
    
    contrast_factor = np.random.uniform(0.8, 1.2)
    print(f"Contrast: {contrast_factor}")
    
    for c in range(X.shape[-1]):
        im = X[:,:,:,c]
        
        brain_im = im[im!=0]
        brain_mean = brain_im.mean()
        
        im = (im - brain_mean) * contrast_factor + brain_mean
        
        X[:,:,:,c] = im
    
    return X, y

def translate(X, y):
    """
    Randomly moves the image by translating it.
    """
    
    # Calculates intensity of translation in each dimension.
    magnitude = 0.1
    x_t = int(magnitude * (X.shape[0]))
    y_t = int(magnitude * (X.shape[1]))
    z_t = int(magnitude * (X.shape[2]))
    
    # Generates random image translation.
    x_t = np.random.randint(-x_t, x_t+1)
    y_t = np.random.randint(-y_t, y_t+1)
    z_t = np.random.randint(-z_t, z_t+1)
    
    print(f"Translate: {x_t}, {y_t}, {z_t}")
    
    # Translates image.
    X = np.roll(X, (x_t, y_t, z_t), axis=(0,1,2))
    y = np.roll(y, (x_t, y_t, z_t), axis=(0,1,2))
    
    # Removes overflow
    backgrounds = X.min(axis=(0,1,2))
    
    for modality in range(X.shape[3]):
        if x_t > 0:
            X[:x_t,:,:,modality] = backgrounds[modality]
            y[:x_t,:,:] = 0
        elif x_t < 0:
            X[x_t:,:,:,modality] = backgrounds[modality]
            y[x_t:,:,:] = 0

        if y_t > 0:
            X[:,:y_t,:,modality] = backgrounds[modality]
            y[:,:y_t,:] = 0
        elif y_t < 0:
            X[:,y_t:,:,modality] = backgrounds[modality]
            y[:,y_t:,:] = 0 

        if z_t > 0:
            X[:,:,:z_t,modality] = backgrounds[modality]
            y[:,:,:z_t] = 0
        elif z_t < 0:
            X[:,:,z_t:,modality] = backgrounds[modality]
            y[:,:,z_t:] = 0
        
    return X,y


def zoom(X, y):
    scaling = np.random.uniform(0.8,1.2)
    print(f"Zoom: {scaling}")
    
    R = np.eye(3) * scaling
    
    X_rot = np.zeros_like(X)
    for channel in range(X.shape[-1]):
        X_rot[:,:,:,channel] = affine_transform(X[:,:,:,channel], R, order=0)
    y_rot = affine_transform(y, R, order=0)
    
    return X_rot, y_rot
    

def shear(X, y):
    
    factor = 0.05
    x_shear = np.random.uniform(-factor, factor)
    y_shear = np.random.uniform(-factor, factor)
    z_shear = np.random.uniform(-factor, factor)
    print(f"Shear: {x_shear},{y_shear},{z_shear}")
    
    shear_mat = np.array([
        [1, y_shear, z_shear],
        [x_shear, 1, z_shear],
        [x_shear, y_shear, 1]
    ])
    
    X_shear = np.zeros_like(X)
    
    backgrounds = X.min(axis=(0,1,2))
    
    for channel in range(X.shape[-1]):
        X_shear[:,:,:,channel] = affine_transform(X[:,:,:,channel], shear_mat, order=0, cval=backgrounds[channel])
        
    y_shear = affine_transform(y, shear_mat, order=0, mode="nearest")
    return X_shear, y_shear

In [183]:
# Defines different data augmentation approaches.

def extreme_augmentation(im, gt):
    "Calculates the most extreme data augmentation for the scan."
    
    im, gt = elastic(im, gt)
    im, gt = flip3D(im, gt)
    im, gt = rotation3D(im, gt)
    im, gt = shear(im, gt)
    im, gt = translate(im, gt)
    im, gt = zoom(im, gt)
    
    im, gt = brightness(im, gt)
    im, gt = contrast(im, gt)
    im, gt = noise(im, gt)
    return im, gt


def balanced_augmentation(im, gt):
    """
    Augments the given datapoint with less extreme transformations than extreme_augmentation(...).
    
    The following brightness-altering augmentations are always applied at the very end:
    - brightness adjust
    - contrast adjust
    - addititive noise
    
    Because of its good performance, every datapoint is augmented with elastic transformation.
    
    Out of the pool of geometric transformation, two are pulled and applied:
    - image flipping
    - rotation
    - translation
    - zoom in/out
    - 3D shearing
    """
    
    # Always applies elastic transformation.
    im, gt = elastic(im, gt)
    
    # Pulls 2 unique random numbers out of range(5).
    choices = np.random.choice(range(5), (conf.num_augmentations,), replace = False)
    
    for choice in choices:
        if choice == 0:
            im, gt = flip3D(im, gt)
        elif choice == 1:
            im, gt = rotation3D(im, gt)
        elif choice == 2:
            im, gt = translate(im, gt)
        elif choice == 3:
            im, gt = zoom(im, gt)
        elif choice == 4:
            im, gt = shear(im, gt)
            
    # Applies all brightness augmentations.
    im, gt = brightness(im, gt)
    im, gt = contrast(im, gt)
    im, gt = noise(im, gt)
    
    return im, gt 

In [184]:
# Optional: Use this to test specific augmentations on a single image.
ENABLED = False

if ENABLED:
    # Export settings:
    folder = conf.aug_export_path + "/examples/" # Specify the export folder here!
    aug_title = "noise" # Name the augmentation you are testing here!
    
    # Loads a single image.
    X, y, patient_id = scan_loader.load_img(data[0], normalize = False)
    
    # Apply the augmentations you want to test here!
    Xaug, yaug = noise(X, y)
    
    # Saves the result.
    scan_name = patient_id + "_" + aug_title
    scan_loader.save_full_scan(Xaug, yaug, folder, scan_name)
    print(f"Done! Exported to: {folder}{scan_name}.")

In [185]:
# Creates data generator
aug_gen = DataGenerator(data, shuffle=False, patch_size=128, n_patches=1, 
                        dim=(240,240,155), preprocessed=False, crop_input=False)

aug_gen = DataGenerator(data,
                        shuffle = False,
                        input_dim = conf.dataset_dim,
                        output_dim = conf.augmented_dim,
                        batch_size = conf.batch_size,
                        n_channels = conf.num_channels,
                        n_classes = conf.num_classes,
                        categorical_classes = False,
                        
)

# Xbatch.shape: num_batches x 128 x 128 x 128 x num_classes
# Ybatch.shape: num_batches x 128 x 128 x 128
for Xbatch, Ybatch, IDbatch in aug_gen:

    # Iterate over all scans in this batch.
    for b in range(Xbatch.shape[0]):

        # Gets one scan from batch.
        im = Xbatch[b,:,:,:,:]
        gt = Ybatch[b,:,:,:]
        patient_id = IDbatch[b]
        
        # Augments scan.
        im_aug, gt_aug = balanced_augmentation(im, gt)
        
        # Crops/extends augmented image.
        if CROP_MODE == "extend":
            im_aug, gt_aug = scan_loader.extend_img(im_aug, gt_aug, OUTPUT_DIM)
        elif CROP_MODE == "crop":
            im_aug, gt_aug = scan_loader.crop_img(im_aug, gt_aug, aug_gen.output_dim, aug_gen.crop_offset)
        
        # Saves augmented scan
        folder = EXPORT_DIR + patient_id + "_aug"
        print(f"Saving {patient_id}_aug...")
        scan_loader.save_full_scan(im_aug, gt_aug, folder, patient_id + "_aug")
        
        # Crops/extends unaugmented image.
        if CROP_MODE == "extend":
            im, gt = scan_loader.extend_img(im, gt, OUTPUT_DIM)
        elif CROP_MODE == "crop":
            im, gt = scan_loader.crop_img(im, gt, aug_gen.output_dim, aug_gen.crop_offset)
        
        # Saves un-augmented scan.
        folder = EXPORT_DIR + patient_id
        print(f"Saving {patient_id}...")
        scan_loader.save_full_scan(im, gt, folder, patient_id)

Elastic: 1.2625836583551764
Rotation: 0.3665191429188092, 0.20943951023931953, 0.47123889803846897
Shear: 0.02853100248343088,-0.010429251143576554,-0.02270898713263041
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_001_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_001...
Elastic: 1.9317362377154854
Shear: 0.03827472538036769,0.04192060012096101,-0.036123101691319506
Rotation: 0.2792526803190927, 0.0, 0.017453292519943295
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_002_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_002...
Elastic: 1.4507125529600398
Zoom: 0.9480464508205775
Flip: 0
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_003_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_003...
Elastic: 1.6655608460309357
Translate: 14, -4, -3
Zoom: 0.8413999413246561
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_004_aug...
(4,) (24

Elastic: 2.7696704358123765
Translate: 9, 15, -9
Flip: 2
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_033_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_033...
Elastic: 0.7089938844310759
Translate: 24, 7, -4
Flip: 2
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_034_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_034...
Elastic: 2.5357264044556165
Flip: 1
Shear: 0.024047471119722302,-0.00517938431227203,0.01133554412432343
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_035_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_035...
Elastic: 2.8907766789436855
Zoom: 1.07093783467696
Rotation: 0.05235987755982988, 0.47123889803846897, 0.296705972839036
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_036_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_036...
Elastic: 2.1320616289045695
Flip: 1
Translate: -8, 6

Zoom: 1.1378638955259335
Translate: 3, -18, 11
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_065_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_065...
Elastic: 0.9688951835339648
Shear: 0.0217905762475811,-0.0462531015897268,-0.005965595395205046
Zoom: 1.1990109439356242
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_066_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_066...
Elastic: 0.6601188088758212
Rotation: 0.15707963267948966, 0.0, 0.08726646259971647
Flip: 2
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_067_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_067...
Elastic: 1.3089514980227297
Rotation: 0.3839724354387525, 0.17453292519943295, 0.03490658503988659
Shear: -0.01995191737085822,0.008223605430447554,0.04931725666749154
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_068_aug...
(4,) (240, 240, 155, 4) (240, 240, 160

Shear: 0.0039189884486719034,0.02552877888577186,0.02886944549900966
Zoom: 1.148853335667789
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_097_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_097...
Elastic: 0.7881980186025768
Rotation: 0.296705972839036, 0.4886921905584123, 0.2792526803190927
Zoom: 1.072389762739714
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_098_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_098...
Elastic: 1.214676103983353
Shear: 0.01952978071940191,0.028110429457098463,-0.039646137162736045
Flip: 0
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_099_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_099...
Elastic: 0.975258871880631
Rotation: 0.12217304763960307, 0.19198621771937624, 0.22689280275926282
Shear: -0.006867540562229234,0.034474492152144265,-0.014749496402113947
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS

(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_128...
Elastic: 1.848170043907332
Flip: 0
Zoom: 1.178627126532112
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_129_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_129...
Elastic: 1.105013801589187
Shear: -0.04386988779306599,0.02453446370434814,0.027417049490206055
Flip: 1
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_130_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_130...
Elastic: 3.6451646813525973
Zoom: 0.9411464214268195
Translate: 0, -24, 4
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_131_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_131...
Elastic: 1.416519102451586
Translate: 4, 20, -9
Flip: 0
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_132_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_132...
Elastic: 2.41889159931335

...

Saving BraTS20_Training_255...
Elastic: 3.428042856295116
Translate: 21, 2, 4
Rotation: 0.296705972839036, 0.2617993877991494, 0.17453292519943295
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_256_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_256...
Elastic: 3.3105053769846458
Rotation: 0.0, 0.4886921905584123, 0.17453292519943295
Zoom: 1.0822060595059502
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_257_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_257...
Elastic: 0.10651449817539538
Rotation: 0.22689280275926282, 0.03490658503988659, 0.47123889803846897
Shear: -0.049264782442365734,-0.010552025639711901,-0.023652248056470183
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_258_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_258...
Elastic: 0.21433845060635637
Translate: 1, 10, -15
Zoom: 1.1980794849060445
(4,) (240, 240, 155, 4) (240, 240, 160,

Saving BraTS20_Training_287...
Elastic: 0.9170858869080165
Shear: -0.048049231477123394,0.017620945741515834,0.013152882997403081
Flip: 2
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_288_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_288...
Elastic: 2.192950688694845
Flip: 2
Zoom: 1.0749883975428607
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_289_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_289...
Elastic: 2.1791574322187395
Zoom: 0.8240976248814837
Flip: 2
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_290_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_290...
Elastic: 0.3187327353154523
Translate: 23, -15, 3
Zoom: 0.9966468949758189
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_291_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_291...
Elastic: 2.921289004399836
Shear: 0.00039597111949554037,-

(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_320_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_320...
Elastic: 0.061087852793870745
Flip: 2
Rotation: 0.03490658503988659, 0.40142572795869574, 0.4886921905584123
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_321_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_321...
Elastic: 2.399134180374495
Zoom: 0.9498471001497935
Rotation: 0.15707963267948966, 0.41887902047863906, 0.3839724354387525
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_322_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_322...
Elastic: 1.4240887348878672
Flip: 1
Zoom: 1.085302531981339
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_323_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_323...
Elastic: 0.9401927336153824
Flip: 0
Shear: -0.0076866200175286364,-0.02182364040629453,-0.0435712241

Flip: 1
Rotation: 0.03490658503988659, 0.4363323129985824, 0.5235987755982988
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_352_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_352...
Elastic: 1.07238021276754
Shear: 0.033362475248500875,0.031208631491546887,-0.029625635261010853
Flip: 2
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_353_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_353...
Elastic: 0.2925931041456429
Translate: 5, -12, 9
Rotation: 0.20943951023931953, 0.10471975511965977, 0.47123889803846897
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_354_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_354...
Elastic: 2.965537753819217
Rotation: 0.03490658503988659, 0.2617993877991494, 0.4363323129985824
Translate: 24, 15, -7
(4,) (240, 240, 155, 4) (240, 240, 160, 4)
Saving BraTS20_Training_355_aug...
(4,) (240, 240, 155, 4) (240, 240, 160, 4)


IndexError: list index out of range